<a href="https://colab.research.google.com/github/plaban1981/NLP_Transformers/blob/main/Copy_of_Lecture_1_Keyword_extraction_and_similarity_search_with_vector_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
movie_plots_path = "/content/drive/MyDrive/NLP_RAM/Week 2/American_Movie_Plots_2005_2021_v2.csv"

In [3]:
import pandas as pd 
df = pd.read_csv(movie_plots_path) 
print (df.shape)
df.head()

(3349, 4)


,Unnamed: 0,Movie Name,Wiki Link,Plot
0,0,White Noise,https://en.wikipedia.org//wiki/White_Noise_(20...,Jonathan Rivers is an architect and lives with...
1,1,Coach Carter,https://en.wikipedia.org//wiki/Coach_Carter,"Ken Carter lives in Richmond, California. He b..."
2,2,Elektra,https://en.wikipedia.org//wiki/Elektra_(2005_f...,"After being killed in Daredevil, Elektra Natch..."
3,3,Racing Stripes,https://en.wikipedia.org//wiki/Racing_Stripes,"During a thunderstorm, a traveling circus, Cir..."
4,4,Tom and Jerry: Blast Off to Mars,https://en.wikipedia.org//wiki/Tom_and_Jerry:_...,Tom (voiced by Bill Kopp) chases Jerry as usua...


In [4]:
print (df.shape)
df.drop_duplicates(subset ="Wiki Link", keep = False, inplace = True)
print (df.shape)

(3349, 4)
(3321, 4)


In [5]:
batmanbegins_df  = df[df['Movie Name'].str.match('Interstellar')]
batmanbegins_plot = batmanbegins_df.iloc[0]['Plot']
print (batmanbegins_plot)

In 2067, crop blights and dust storms threaten humanity's survival. Cooper, a widowed engineer and former NASA pilot turned farmer, lives with his father-in-law, Donald, his 15-year-old son, Tom, and 10-year-old daughter, Murphy "Murph". After a dust storm, strange dust patterns inexplicably appear in Murphy's bedroom; she attributes the anomaly to a ghost. Cooper eventually deduces the patterns were caused by gravity variations and they represent geographic coordinates in binary code. Cooper follows the coordinates to a secret NASA facility headed by Professor John Brand.
48 years earlier, unknown beings positioned a wormhole near Saturn, opening a path to a distant galaxy with 12 potentially habitable worlds located near a black hole named Gargantua. Twelve volunteers traveled through the wormhole to individually survey the planets and three — Dr. Mann, Laura Miller, and Wolf Edmunds — reported positive results. Based on their data, Professor Brand conceived two plans to ensure human

# Extract Keywords

In [1]:
!python3 -m spacy download en_core_web_md

     |████████████████████████████████| 96.4 MB 1.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051302 sha256=75e833095de560379349a597330b9507db7ffdefc16c19532902ac40bf00eb98
  Stored in directory: /tmp/pip-ephem-wheel-cache-gpbkezu1/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


# Restart runtime after spacy download

In [6]:
import spacy
nlp = spacy.load('en_core_web_md')

def get_noun_phrases(sample_text):
  candidates =[]
  doc = nlp(sample_text)
  for np in doc.noun_chunks:#extract all noun phrases in the text
    phrase = np.text
    # no_of_words = phrase.split()
    # if len(no_of_words)>=2 and len(no_of_words)<=4:
    #   phrase = np.text
    phrase = phrase.strip()
    if phrase not in candidates:
      candidates.append(phrase)
  return candidates

print (get_noun_phrases(batmanbegins_plot))

['crop blights', 'dust storms', "humanity's survival", 'Cooper', 'a widowed engineer', 'former NASA pilot', 'farmer', 'law', 'Donald', 'his 15-year-old son', 'Tom', '10-year-old daughter', 'a dust storm', 'strange dust patterns', "Murphy's bedroom", 'she', 'the anomaly', 'a ghost', 'the patterns', 'gravity variations', 'they', 'geographic coordinates', 'binary code', 'the coordinates', 'a secret NASA facility', 'Professor John Brand', 'unknown beings', 'a wormhole', 'Saturn', 'a path', 'a distant galaxy', '12 potentially habitable worlds', 'a black hole', 'Twelve volunteers', 'the wormhole', 'the planets', 'Dr. Mann', 'Laura Miller', 'Wolf Edmunds', 'positive results', 'their data', 'Professor Brand', 'two plans', 'Plan A', 'a gravitational propulsion theory', 'settlements', 'space', 'Plan B', 'the Endurance spacecraft', '5,000 frozen human embryos', 'a habitable planet', 'the Endurance', 'a distraught Murphy', 'his wristwatch', 'their relative time', 'he', 'Dr. Romilly', 'the black ho

In [ ]:
# !pip install transformers==4.5.1
# !pip install keybert==0.4.0

In [7]:
np_batman_begins = get_noun_phrases(batmanbegins_plot)
print (np_batman_begins)

['crop blights', 'dust storms', "humanity's survival", 'Cooper', 'a widowed engineer', 'former NASA pilot', 'farmer', 'law', 'Donald', 'his 15-year-old son', 'Tom', '10-year-old daughter', 'a dust storm', 'strange dust patterns', "Murphy's bedroom", 'she', 'the anomaly', 'a ghost', 'the patterns', 'gravity variations', 'they', 'geographic coordinates', 'binary code', 'the coordinates', 'a secret NASA facility', 'Professor John Brand', 'unknown beings', 'a wormhole', 'Saturn', 'a path', 'a distant galaxy', '12 potentially habitable worlds', 'a black hole', 'Twelve volunteers', 'the wormhole', 'the planets', 'Dr. Mann', 'Laura Miller', 'Wolf Edmunds', 'positive results', 'their data', 'Professor Brand', 'two plans', 'Plan A', 'a gravitational propulsion theory', 'settlements', 'space', 'Plan B', 'the Endurance spacecraft', '5,000 frozen human embryos', 'a habitable planet', 'the Endurance', 'a distraught Murphy', 'his wristwatch', 'their relative time', 'he', 'Dr. Romilly', 'the black ho

In [8]:
print (len(np_batman_begins))

146


## Sentence Transformers

In [9]:
!pip install --quiet sentence-transformers==2.0.0

     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 3.1 MB 23.0 MB/s 
     |████████████████████████████████| 1.2 MB 57.6 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 54.9 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 3.3 MB 55.0 MB/s 


In [10]:
from sentence_transformers import SentenceTransformer
model= SentenceTransformer('all-MiniLM-L12-v2')

def get_doc_and_candidate_embeddings(transformermodel,doc_text,candiate_phrases):
  doc_embedding = transformermodel.encode([doc_text])
  candidate_embeddings = transformermodel.encode(candiate_phrases)
  return doc_embedding,candidate_embeddings

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/573 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
document_embedd, keyphrase_embedds = get_doc_and_candidate_embeddings(model,batmanbegins_plot,np_batman_begins)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Extract similarity within words, and between words and the document
word_doc_similarity = cosine_similarity(keyphrase_embedds, document_embedd)
word_doc_similarity = word_doc_similarity.flatten()
# print (word_doc_similarity)
# print (word_doc_similarity.flatten())
# print (word_doc_similarity.shape)

keywords_idx = np.argsort(word_doc_similarity)
keywords_idx = keywords_idx[::-1]

print (word_doc_similarity[keywords_idx])

# print(np_batman_begins[keywords_idx])

[ 3.32219392e-01  2.91050494e-01  2.82631576e-01  2.53399581e-01
  2.42108658e-01  2.41039380e-01  2.11302921e-01  1.93605646e-01
  1.90775186e-01  1.55727535e-01  1.47730678e-01  1.47589371e-01
  1.33162826e-01  1.30303726e-01  1.20684341e-01  1.17184319e-01
  1.15216397e-01  1.14736825e-01  1.13116145e-01  1.12669438e-01
  1.07783340e-01  1.07317097e-01  1.06043547e-01  1.05295748e-01
  1.04457244e-01  9.49720517e-02  9.47955772e-02  8.64329040e-02
  8.21129680e-02  7.96528682e-02  7.86606371e-02  7.82323033e-02
  7.64017850e-02  7.53204599e-02  7.33562633e-02  7.17613772e-02
  6.96512312e-02  6.75772578e-02  6.75014183e-02  6.73745647e-02
  6.68542758e-02  6.67967796e-02  6.52762651e-02  5.83589561e-02
  5.83589561e-02  5.78797720e-02  5.70534877e-02  5.67470305e-02
  5.46454564e-02  5.33924587e-02  4.90123332e-02  4.59267274e-02
  4.45816070e-02  4.31095399e-02  4.25839126e-02  4.20546643e-02
  3.88421901e-02  3.85827981e-02  3.59794721e-02  3.54214497e-02
  3.31709422e-02  3.02148

In [18]:
sorted_keywords =[]

for index in keywords_idx:
  sorted_keywords.append(np_batman_begins[index])

print (sorted_keywords)

['the younger Murphy', 'Murphy', 'a distraught Murphy', 'Murphy\'s "ghost', "Murphy's old room", "Murphy's bedroom", 'an elderly Murphy', 'his 15-year-old son', 'the singularity', 'Cooper', 'Cooper’s order', 'death', 'a black hole', 'a dust storm', 'cryostasis', '23 years', "Doyle's death", 'the "ghost', 'a ghost', 'survival', 'heads', 'Morse code', 'the black hole', 'the Morse code', 'dust storms', 'law', '10-year-old daughter', 'her own family', 'Tom', 'control', 'the process', 'falsified data', 'farmer', 'it', 'the aberrant ticking', 'him', 'another 51 years', 'his wristwatch', 'Brand and Cooper race', 'strange dust patterns', 'a booby trap', 'the wreckage', 'time', 'he', 'He', 'the quantum data', 'the event horizon', 'Wolf Edmunds', "Miller's planet", 'the time', "Gargantua's proximity", 'Laura Miller', 'dilation', 'a difficult docking manoeuver', 'a failed docking operation', 'a slingshot maneuver', 'the second hand', "humanity's survival", 'Plan A', "Mann's planet", '5,000 frozen

## https://github.com/MaartenGr/KeyBERT

In [ ]:
from typing import List, Tuple
import itertools
def max_sum_similarity(doc_embedding: np.ndarray,
                       word_embeddings: np.ndarray,
                       words: List[str],
                       top_n: int,
                       nr_candidates: int) -> List[Tuple[str, float]]:
    """ Calculate Max Sum Distance for extraction of keywords
    We take the 2 x top_n most similar words/phrases to the document.
    Then, we take all top_n combinations from the 2 x top_n words and
    extract the combination that are the least similar to each other
    by cosine similarity.
    NOTE:
        This is O(n^2) and therefore not advised if you use a large top_n
    Arguments:
        doc_embedding: The document embeddings
        word_embeddings: The embeddings of the selected candidate keywords/phrases
        words: The selected candidate keywords/keyphrases
        top_n: The number of keywords/keyhprases to return
        nr_candidates: The number of candidates to consider
    Returns:
         List[Tuple[str, float]]: The selected keywords/keyphrases with their distances
    """
    if nr_candidates < top_n:
        raise Exception("Make sure that the number of candidates exceeds the number "
                        "of keywords to return.")

    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, word_embeddings)
    distances_words = cosine_similarity(word_embeddings, word_embeddings)

    # Get 2*top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [words[index] for index in words_idx]
    candidates = distances_words[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = 100_000
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [(words_vals[idx], round(float(distances[0][idx]), 4)) for idx in candidate]


print ( max_sum_similarity(document_embedd,keyphrase_embedds,np_batman_begins,6,20))

[('the Morse code', 0.3459), ('his 15-year-old son', 0.1858), ('Brand and Cooper race', 0.5237), ('strange dust patterns', 0.2625), ('a distraught Murphy', 0.2814), ('a secret NASA facility', 0.1566)]


# Find Similar Documents

In [11]:
df.head()

,Unnamed: 0,Movie Name,Wiki Link,Plot
0,0,White Noise,https://en.wikipedia.org//wiki/White_Noise_(20...,Jonathan Rivers is an architect and lives with...
1,1,Coach Carter,https://en.wikipedia.org//wiki/Coach_Carter,"Ken Carter lives in Richmond, California. He b..."
2,2,Elektra,https://en.wikipedia.org//wiki/Elektra_(2005_f...,"After being killed in Daredevil, Elektra Natch..."
3,3,Racing Stripes,https://en.wikipedia.org//wiki/Racing_Stripes,"During a thunderstorm, a traveling circus, Cir..."
4,4,Tom and Jerry: Blast Off to Mars,https://en.wikipedia.org//wiki/Tom_and_Jerry:_...,Tom (voiced by Bill Kopp) chases Jerry as usua...


In [12]:
all_plots = df['Plot'].tolist()
print (len(all_plots))

3321


In [13]:
batmanbegins_df  = df[df['Movie Name'].str.match('Batman Begins')]
batmanbegins_plot = batmanbegins_df.iloc[0]['Plot']
print (batmanbegins_plot)

In Gotham City, young Bruce Wayne falls down a dry well and is attacked by a swarm of bats, developing a fear of them. Attending the opera with his parents, Thomas and Martha, Bruce becomes frightened by performers masquerading as bats and asks to leave. Outside, mugger Joe Chill murders Bruce's parents in front of him, and the orphaned Bruce is raised by the family butler, Alfred Pennyworth.
Fourteen years later, Chill is paroled after testifying against mafia boss Carmine Falcone. Bruce intends to murder Chill to avenge his parents, but one of Falcone's assassins does so first. Bruce's childhood friend Rachel Dawes berates him for acting outside the justice system, saying that his father would be ashamed. After confronting Falcone, who tells him that real power comes from being feared, Bruce spends the next seven years traveling the world training in combat and immersing himself in the criminal underworld. In a Bhutan prison, he meets Henri Ducard, who recruits him to the League of S

In [14]:
document_embedd, plot_embedds = get_doc_and_candidate_embeddings(model,batmanbegins_plot,all_plots)

In [23]:
word_doc_similarity = cosine_similarity(plot_embedds, document_embedd)
word_doc_similarity = word_doc_similarity.flatten()

plots_idx = np.argsort(word_doc_similarity)
plots_idx = plots_idx[::-1]

sorted_plots =[]
sorted_titles=[]
scores = [] 

all_titles=df['Movie Name'].tolist()

for index in plots_idx:
  sorted_plots.append(all_plots[index])
  sorted_titles.append(all_titles[index])
  scores.append(word_doc_similarity[index])


N = 6 

for i,plot,title in zip(scores[:N],sorted_plots[:N],sorted_titles[:N]):
  print (f'Movie Title : {title}')
  print (f'Movie Plot :{plot}')
  print(f'Similarity Score :{i}')
  print ("\n\n")


Movie Title : Batman Begins
Movie Plot :In Gotham City, young Bruce Wayne falls down a dry well and is attacked by a swarm of bats, developing a fear of them. Attending the opera with his parents, Thomas and Martha, Bruce becomes frightened by performers masquerading as bats and asks to leave. Outside, mugger Joe Chill murders Bruce's parents in front of him, and the orphaned Bruce is raised by the family butler, Alfred Pennyworth.
Fourteen years later, Chill is paroled after testifying against mafia boss Carmine Falcone. Bruce intends to murder Chill to avenge his parents, but one of Falcone's assassins does so first. Bruce's childhood friend Rachel Dawes berates him for acting outside the justice system, saying that his father would be ashamed. After confronting Falcone, who tells him that real power comes from being feared, Bruce spends the next seven years traveling the world training in combat and immersing himself in the criminal underworld. In a Bhutan prison, he meets Henri Duc